In [12]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
from tabulate import tabulate

# Load environment variables from .env file
load_dotenv("../Credential/.env")

# Get the USDA API key from the environment variable
USDA_API_KEY = os.getenv("USDA_API_KEY")

# Function#1.1 to fetch food details from USDA API given an FDC ID
def fetch_food_details(fdc_id, api_key=USDA_API_KEY):
    url = f"https://api.nal.usda.gov/fdc/v1/food/{fdc_id}?api_key={api_key}"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error fetching food details. Status code: {response.status_code}")
        return None
    try:
        return response.json()
    except Exception as e:
        print(f"Error decoding JSON response: {e}")

# Function#1.2 to convert the "food_details" variable to a Markdown table
def food_details_to_markdown(food_details):
    # Fetch the food name from the API response and keep only the first part before the comma
    full_food_name = food_details.get('description', 'Unknown')
    food_name = full_food_name.split(',')[0]
    
    food_nutrients_list = food_details.get('foodNutrients', [])
    df_food_nutrients = pd.DataFrame(columns=['Nutrient', 'Unit', food_name])
    
    for item in food_nutrients_list:
        nutrient_name = item.get('nutrient', {}).get('name', 'Unknown')
        amount = item.get('amount', 'N/A')
        unit = item.get('nutrient', {}).get('unitName', 'N/A')
        
        new_row = pd.DataFrame({'Nutrient': [nutrient_name], food_name: [amount], 'Unit': [unit]})
        df_food_nutrients = pd.concat([df_food_nutrients, new_row], ignore_index=True)
    
    markdown_table = tabulate(df_food_nutrients, tablefmt="pipe", headers="keys")
    
    return markdown_table

# Example usage
fdc_id = 168428  # Replace with a valid FDC ID
food_details = fetch_food_details(fdc_id)

if food_details is not None:
    markdown_table_from_food_details = food_details_to_markdown(food_details)
    print(markdown_table_from_food_details)

|    | Nutrient                                 | Unit   | Lentils   |
|---:|:-----------------------------------------|:-------|:----------|
|  0 | Proximates                               | g      | N/A       |
|  1 | Water                                    | g      | 68.7      |
|  2 | Energy                                   | kcal   | 101.0     |
|  3 | Energy                                   | kJ     | 423.0     |
|  4 | Protein                                  | g      | 8.8       |
|  5 | Total lipid (fat)                        | g      | 0.45      |
|  6 | Ash                                      | g      | 0.8       |
|  7 | Carbohydrates                            | g      | N/A       |
|  8 | Carbohydrate, by difference              | g      | 21.25     |
|  9 | Minerals                                 | mg     | N/A       |
| 10 | Calcium, Ca                              | mg     | 14.0      |
| 11 | Iron, Fe                                 | mg     | 3.1       |
| 12 |

In [15]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
from tabulate import tabulate

# Load environment variables from .env file
load_dotenv("../Credential/.env")

# Get the USDA API key from the environment variable
USDA_API_KEY = os.getenv("USDA_API_KEY")

# Function#1 to fetch food details from USDA API given an FDC ID
def fetch_food_details(fdc_id, api_key=USDA_API_KEY):
    url = f"https://api.nal.usda.gov/fdc/v1/food/{fdc_id}?api_key={api_key}"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error fetching food details. Status code: {response.status_code}")
        return None
    try:
        return response.json()
    except Exception as e:
        print(f"Error decoding JSON response: {e}")
        return None

# Function#2 to fetch and scale nutrients for a single food item
def fetch_and_scale_nutrients(fdc_id, grams):
    food_details = fetch_food_details(fdc_id)
    if food_details is None:
        return None
    
    food_name = food_details.get('description', 'Unknown').split(',')[0]
    food_nutrients_list = food_details.get('foodNutrients', [])
    
    df_food_nutrients = pd.DataFrame(columns=['Nutrient', 'Unit', food_name])
    
    for item in food_nutrients_list:
        nutrient_name = item.get('nutrient', {}).get('name', 'Unknown')
        amount_per_100g = item.get('amount', 'N/A')
        unit = item.get('nutrient', {}).get('unitName', 'N/A')
        
        try:
            # Attempt to convert the amount to float and scale it
            scaled_amount = (float(amount_per_100g) * grams) / 100.0
            scaled_amount = round(scaled_amount, 2)  # Round to two decimal places
        except ValueError:
            # If conversion fails, set scaled_amount to 0
            scaled_amount = 0.0
        
        new_row = pd.DataFrame({'Nutrient': [nutrient_name], food_name: [scaled_amount], 'Unit': [unit]})
        df_food_nutrients = pd.concat([df_food_nutrients, new_row], ignore_index=True)
    
    return df_food_nutrients


# Function#3 to aggregate nutrients from multiple food items
def aggregate_nutrients(df_list):
    df_aggregated = pd.concat(df_list, ignore_index=True)
    
    # Group by 'Nutrient' and 'Unit', and sum the nutrient values
    df_aggregated = df_aggregated.groupby(['Nutrient', 'Unit']).sum().reset_index()
    
    # Replace 'N/A' with 0.0 for numerical operations
    df_aggregated.replace('N/A', 0.0, inplace=True)
    
    # Create a 'Sum' column that sums up the nutrient values for each nutrient type
    df_aggregated['Sum'] = df_aggregated.iloc[:, 2:].sum(axis=1).astype('float64').round(2)
    
    # Reorder the columns to place 'Sum' before each individual food column
    cols = df_aggregated.columns.tolist()
    reordered_cols = cols[:2] + ['Sum'] + cols[2:-1]
    df_aggregated = df_aggregated[reordered_cols]
    
    return df_aggregated

# Main function
def main():
    food_list = [
        {"Food": "Raw Sprouted", "fdc_id":  168427, "gram": 100},
        {"Food": "Cooked Sprouted", "fdc_id": 173950, "gram": 100}, #BUG ALERT! Side-by-side comparison is fail e.g. 168427 & 168428
    ]
    
    df_list = []
    
    for food_item in food_list:
        df = fetch_and_scale_nutrients(food_item['fdc_id'], food_item['gram'])
        if df is not None:
            df_list.append(df)
    
    df_aggregated = aggregate_nutrients(df_list)
    
    # Export the aggregated DataFrame to a CSV file
    #df_aggregated.to_csv("aggregated_nutrients.csv", index=False)
    #print("Aggregated nutrients data has been exported to 'aggregated_nutrients.csv'.")
    
    print(tabulate(df_aggregated, tablefmt="plain", headers="keys"))

# Run the main function
if __name__ == "__main__":
    main()

    Nutrient                                  Unit       Sum    Lentils    Blueberries
 0  Alanine                                   g         0.38       0.36           0.02
 1  Alcohol, ethyl                            g         0          0              0
 2  Amino acids                               g         0          0              0
 3  Arginine                                  g         0.63       0.61           0.02
 4  Ash                                       g         1.18       1              0.18
 5  Aspartic acid                             g         1.46       1.43           0.03
 6  Caffeine                                  mg        0          0              0
 7  Calcium, Ca                               mg       33         25              8
 8  Carbohydrate, by difference               g        34.31      22.14          12.17
 9  Carbohydrates                             g         0          0              0
10  Carotene, alpha                           µg        0 

In [ ]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
from tabulate import tabulate
import matplotlib.pyplot as plt

# Load environment variables from .env file
load_dotenv("../Credential/.env")

# Get the USDA API key from the environment variable
USDA_API_KEY = os.getenv("USDA_API_KEY")

# Define the categories and their corresponding nutrients
categories = {
    "AAFCO_CC_Protein": [
        {"Nutrient": "Arginine", "unit": "g"},
        {"Nutrient": "Histidine", "unit": "g"},
        {"Nutrient": "Isoleucine", "unit": "g"},
        {"Nutrient": "Leucine", "unit": "g"},
        {"Nutrient": "Lysine", "unit": "g"},
        {"Nutrient": "Methionine", "unit": "g"},
        {"Nutrient": "Cystine", "unit": "g"},
        {"Nutrient": "Phenylalanine", "unit": "g"},
        {"Nutrient": "Tyrosine", "unit": "g"},
        {"Nutrient": "Threonine", "unit": "g"},
        {"Nutrient": "Tryptophan", "unit": "g"},
        {"Nutrient": "Valine", "unit": "g"}
    ],
    "AAFCO_CC_Fat": [
        {"Nutrient": "PUFA 18:3 n-3 c,c,c (ALA)", "unit": "g"},
        {"Nutrient": "PUFA 18:2 n-6 c,c", "unit": "g"},
        {"Nutrient": "PUFA 20:5 n-3 (EPA)", "unit": "g"},
        {"Nutrient": "PUFA 22:6 n-3 (DHA)", "unit": "g"}
    ],
    "AAFCO_CC_Mineral": [
        {"Nutrient": "Calcium, Ca", "unit": "g"},
        {"Nutrient": "Phosphorus, P", "unit": "g"},
        {"Nutrient": "Potassium, K", "unit": "g"},
        {"Nutrient": "Sodium, Na", "unit": "g"},
        {"Nutrient": "Chlorine, Cl", "unit": "g"},
        {"Nutrient": "Magnesium, Mg", "unit": "g"},
        {"Nutrient": "Iron, Fe", "unit": "mg"},
        {"Nutrient": "Copper, Cu", "unit": "mg"},
        {"Nutrient": "Manganese, Mn", "unit": "mg"},
        {"Nutrient": "Zinc, Zn", "unit": "mg"},
        {"Nutrient": "Iodine, I", "unit": "mg"},
        {"Nutrient": "Selenium, Se", "unit": "mg"}
    ],
    "AAFCO_CC_Vitamins": [
        {"Nutrient": "Vitamin D (D2 + D3), International Units", "unit": "IU"},
        {"Nutrient": "Vitamin E (alpha-tocopherol)", "unit": "IU"},
        {"Nutrient": "Thiamin", "unit": "mg"},
        {"Nutrient": "Riboflavin", "unit": "mg"},
        {"Nutrient": "Pantothenic acid", "unit": "mg"},
        {"Nutrient": "Niacin", "unit": "mg"},
        {"Nutrient": "Vitamin B-6", "unit": "mg"},
        {"Nutrient": "Folic acid", "unit": "mg"},
        {"Nutrient": "Vitamin B-12", "unit": "mg"},
        {"Nutrient": "Choline, total", "unit": "mg"}
    ]
}
    
# Function#1 to fetch food details from USDA API given an FDC ID
def fetch_food_details(fdc_id, api_key=USDA_API_KEY):
    url = f"https://api.nal.usda.gov/fdc/v1/food/{fdc_id}?api_key={api_key}"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error fetching food details. Status code: {response.status_code}")
        return None
    try:
        return response.json()
    except Exception as e:
        print(f"Error decoding JSON response: {e}")
        return None

# Function#2 to fetch and scale nutrients for a single food item
def fetch_and_scale_nutrients(fdc_id, grams):
    food_details = fetch_food_details(fdc_id)
    if food_details is None:
        return None
    
    food_name = food_details.get('description', 'Unknown').split(',')[0]
    food_nutrients_list = food_details.get('foodNutrients', [])
    
    df_food_nutrients = pd.DataFrame(columns=['Nutrient', 'Unit', food_name])
    
    for item in food_nutrients_list:
        nutrient_name = item.get('nutrient', {}).get('name', 'Unknown')
        amount_per_100g = item.get('amount', 'N/A')
        unit = item.get('nutrient', {}).get('unitName', 'N/A')
        
        if amount_per_100g != 'N/A':
            scaled_amount = (amount_per_100g * grams) / 100.0
            scaled_amount = round(scaled_amount, 2)  # Round to two decimal places
        else:
            scaled_amount = 'N/A'
        
        new_row = pd.DataFrame({'Nutrient': [nutrient_name], food_name: [scaled_amount], 'Unit': [unit]})
        df_food_nutrients = pd.concat([df_food_nutrients, new_row], ignore_index=True)
    
    return df_food_nutrients

# Function#3 to aggregate nutrients from multiple food items
def aggregate_nutrients(df_list):
    df_aggregated = pd.concat(df_list, ignore_index=True)
    
    # Group by 'Nutrient' and 'Unit', and sum the nutrient values
    df_aggregated = df_aggregated.groupby(['Nutrient', 'Unit']).sum().reset_index()
    
    # Replace 'N/A' with 0.0 for numerical operations
    df_aggregated.replace('N/A', 0.0, inplace=True)
    
    # Create a 'Sum' column that sums up the nutrient values for each nutrient type
    df_aggregated['Sum'] = df_aggregated.iloc[:, 2:].sum(axis=1).astype('float64').round(2)
    
    # Reorder the columns to place 'Sum' before each individual food column
    cols = df_aggregated.columns.tolist()
    reordered_cols = cols[:2] + ['Sum'] + cols[2:-1]
    df_aggregated = df_aggregated[reordered_cols]
    
    return df_aggregated

# Function#4 to plot nutrient density for each category
def plot_nutrient_density(df_aggregated, categories):
    for category, nutrient_list in categories.items():
        
        # Create a DataFrame with all nutrients of interest initialized to zero
        all_nutrients_df = pd.DataFrame(
            [{'Nutrient': item['Nutrient'], 'Unit': item['unit']} for item in nutrient_list]
        )
        for food in df_aggregated.columns[3:-1]:  # Exclude 'Sum' from the columns
            all_nutrients_df[food] = 0.0

        # Update with actual values
        df_filtered = df_aggregated[df_aggregated.apply(lambda row: any((row['Nutrient'] == item['Nutrient'] and row['Unit'] == item['unit']) for item in nutrient_list), axis=1)]
        all_nutrients_df.update(df_filtered)
        
        plt.figure(figsize=(14, 8))
        
        for food in all_nutrients_df.columns[2:]:  # Exclude 'Nutrient' and 'Unit'
            plt.barh(all_nutrients_df['Nutrient'] + ' (' + all_nutrients_df['Unit'] + ')', all_nutrients_df[food], alpha=0.7, label=food)
        
        plt.title(f"{category} Density")
        plt.ylabel("Nutrient (Unit)")
        plt.xlabel("Amount")
        plt.legend()
        plt.tight_layout()
        plt.show()


# Add the new Macro_nutrient category to the existing categories dictionary
categories['Macro_nutrient'] = [
    {"Nutrient": "Energy", "unit": "Kcal"},
    {"Nutrient": "Water", "unit": "g"},
    {"Nutrient": "Carbohydrate, by difference", "unit": "g"},
    {"Nutrient": "Protein", "unit": "g"},
    {"Nutrient": "Total lipid (fat)", "unit": "g"}
]

# Function to display a summary table of macronutrients
def display_macro_nutrient_summary(df_aggregated, categories):
    macro_nutrients = categories['Macro_nutrient']
    df_macro_nutrients = df_aggregated[df_aggregated.apply(lambda row: any((row['Nutrient'] == item['Nutrient'] and row['Unit'] == item['unit']) for item in macro_nutrients), axis=1)]
    print("Summary of Macronutrients:")
    print(tabulate(df_macro_nutrients, tablefmt="plain", headers="keys")) 

# Function to plot nutrient density for each category, including Macro_nutrients
def plot_nutrient_density(df_aggregated, categories):
    for category, nutrient_list in categories.items():
        
        # Create a DataFrame with all nutrients of interest initialized to zero
        all_nutrients_df = pd.DataFrame(
            [{'Nutrient': item['Nutrient'], 'Unit': item['unit']} for item in nutrient_list]
        )
        for food in df_aggregated.columns[3:]:  # Include 'Sum' and all other columns that contain food names
            all_nutrients_df[food] = 0.0

        # Update with actual values
        df_filtered = df_aggregated[df_aggregated.apply(lambda row: any((row['Nutrient'] == item['Nutrient'] and row['Unit'] == item['unit']) for item in nutrient_list), axis=1)]
        all_nutrients_df.update(df_filtered)
        
        plt.figure(figsize=(14, 8))
        
        for food in all_nutrients_df.columns[2:]:  # Exclude 'Nutrient' and 'Unit'
            plt.barh(all_nutrients_df['Nutrient'] + ' (' + all_nutrients_df['Unit'] + ')', all_nutrients_df[food], alpha=0.7, label=food)
        
        plt.title(f"{category} Density")
        plt.ylabel("Nutrient (Unit)")
        plt.xlabel("Amount")
        plt.legend()
        plt.tight_layout()
        plt.show()

# ...

# Main function
def main():
    food_list = [
        {"Food": "Blueberries", "fdc_id": 173950, "gram": 100},
        {"Food": "Broccoli", "fdc_id": 170379, "gram": 100},
    ]
    
    df_list = []
    
    for food_item in food_list:
        df = fetch_and_scale_nutrients(food_item['fdc_id'], food_item['gram'])
        if df is not None:
            df_list.append(df)
    
    df_aggregated = aggregate_nutrients(df_list)
    
    # Display the aggregated DataFrame as a table
    print("Aggregated Nutrients:")
    print(tabulate(df_aggregated, tablefmt="plain", headers="keys"))
    
    # Plot the nutrient density for each category
    for category, nutrient_list in categories.items():
        
        # Create a DataFrame with all nutrients of interest initialized to zero
        all_nutrients_df = pd.DataFrame(
            [{'Nutrient': item['Nutrient'], 'Unit': item['unit']} for item in nutrient_list]
        )
        for food in df_aggregated.columns[3:-1]:  # Exclude 'Sum' from the columns
            all_nutrients_df[food] = 0.0

        # Update with actual values
        df_filtered = df_aggregated[df_aggregated.apply(lambda row: any((row['Nutrient'] == item['Nutrient'] and row['Unit'] == item['unit']) for item in nutrient_list), axis=1)]
        all_nutrients_df.update(df_filtered)
        
        # Display the filtered DataFrame as a table
        print(f"{category} Nutrients:")
        print(tabulate(all_nutrients_df, tablefmt="plain", headers="keys"))
        
        plt.figure(figsize=(14, 8))
        
        for food in all_nutrients_df.columns[2:]:  # Exclude 'Nutrient' and 'Unit'
            plt.barh(all_nutrients_df['Nutrient'] + ' (' + all_nutrients_df['Unit'] + ')', all_nutrients_df[food], alpha=0.7, label=food)
        
        plt.title(f"{category} Density")
        plt.ylabel("Nutrient (Unit)")
        plt.xlabel("Amount")
        plt.legend()
        plt.tight_layout()
        plt.show()

# Run the main function
if __name__ == "__main__":
    main()
